In [1]:
import sys
import os
import math
import glob
import ROOT as r
import json

# ... fixpath(), style setups, etc.


import ROOT

# --------------------------------------------------------------------
# 1. Set up cross section (pb) and integrated luminosity (1 ab^-1)
#    For example, a background sample with sigma=2.641 pb
# --------------------------------------------------------------------
cross_section_pb = 2.641
luminosity_ab = 1.0  # 1 ab^-1
# Convert 1 ab^-1 to 1e6 pb^-1:
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

# --------------------------------------------------------------------
# 2. Open the ROOT file containing your generated events
# --------------------------------------------------------------------
infile = ROOT.TFile.Open("test_Higgsino_sample.root")
tree = infile.Get("outputTree")  # or "events", depending on your file

# Find total number of generated events, N_gen
N_gen = tree.GetEntries()

# --------------------------------------------------------------------
# 3. Compute the per-event weight
# --------------------------------------------------------------------
weight = (cross_section_pb * luminosity_pb_inv) / N_gen
print(f"Cross section  : {cross_section_pb} pb")
print(f"Luminosity     : {luminosity_ab} ab^-1  ({luminosity_pb_inv} pb^-1)")
print(f"N_gen          : {N_gen}")
print(f"Event weight   : {weight:.3f}")

# --------------------------------------------------------------------
# 4. Create a histogram and fill it with (var, weight)
# --------------------------------------------------------------------
hist = ROOT.TH1F("hist_photon_E", "Photon Energy", 50, 0, 150)  # example
hist.Sumw2()  # keep track of weighted errors

for event in tree:
    for phE in event.ph_E:  # example: an array of photon energies
        hist.Fill(phE, weight)

# --------------------------------------------------------------------
# 5. The histogram now represents the expected yield for that sample 
#    at L = 1 ab^-1 (with the given cross section).
#    Optionally, display or save it.
# --------------------------------------------------------------------
canvas = ROOT.TCanvas("c", "c", 800, 600)
hist.Draw("HIST")
canvas.SaveAs("photon_energy_weighted.png")

infile.Close()


# ---------------------------------------------------------
# 1. Load and fill the background histogram ONCE
# ---------------------------------------------------------
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root")
BGtree = BGfile.Get("outputTree")

hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_Z_BG = r.TH1F("m_Z_BG",      "Z Mass (Background)", 100, 50, 250)
hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)

hist_H_BG.Sumw2()
hist_Z_BG.Sumw2()
hist_photon_energy_BG.Sumw2()

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

for event in BGtree:
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy)
    if len(event.mu_pt) != 2:
        continue
    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])
    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M())
    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M())

# ---------------------------------------------------------
# 2. Process all signal files
# ---------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*.root")

for idx, sig_file in enumerate(signal_files):
    print(f"Processing signal file: {sig_file}")
    
    # Open the signal file
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    
    # Unique name histograms
    hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}", "Z Mass (Signal)", 100, 50, 250)
    hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)
    
    hist_H_signal.Sumw2()
    hist_Z_signal.Sumw2()
    hist_photon_energy_signal.Sumw2()

    # Fill histograms from TTree
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy)

        if len(event.mu_pt) != 2:
            continue
        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])
        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M())
        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M())

    # Now draw them (optional)
    canvas_recoil = r.TCanvas(f"m_recoil_canvas_{idx}", f"Recoil Mass Canvas {idx}", 800, 600)
    hist_H_signal.SetLineColor(r.kRed)
    hist_H_signal.Draw("hist")
    hist_H_BG.SetLineColor(r.kYellow)
    hist_H_BG.Draw("hist same")

    hist_Z_signal.SetLineColor(r.kBlue)
    hist_Z_signal.Draw("hist same")
    hist_Z_BG.SetLineColor(r.kOrange)
    hist_Z_BG.Draw("hist same")

    legend = r.TLegend(0.6, 0.7, 0.85, 0.85)
    legend.AddEntry(hist_H_signal, "Recoil Mass (Signal)", "l")
    legend.AddEntry(hist_H_BG,    "Recoil Mass (BG)", "l")
    legend.AddEntry(hist_Z_signal, "Z Mass (Signal)", "l")
    legend.AddEntry(hist_Z_BG,    "Z Mass (BG)", "l")
    legend.Draw()

    canvas_recoil.SaveAs(f"Recoil_Z_plots_{idx}.png")

    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)
    hist_photon_energy_signal.SetLineColor(r.kGreen)
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # -------------------------------------------------------
    # Optionally, write these histograms to a ROOT file
    # -------------------------------------------------------
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Write background as well (if you want):
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # -------------------------------------------------------
    # Or, export to JSON
    # -------------------------------------------------------
    data_dict = {
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }
    
    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }
    
    for hname, hist in hist_map.items():
        nbins = hist.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = hist.GetXaxis().GetBinCenter(bin_num)
            bin_content = hist.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)
    
    with open(f"extracted_data_{idx}.json", "w") as f:
        json.dump(data_dict, f, indent=4)

    # Finished with this signal file
    infile.Close()

print("All signal files processed.")


Welcome to JupyROOT 6.24/06


OSError: Failed to open file test_Higgsino_sample.root

In [11]:
import sys
import os
import math
import glob
import json
import ROOT as r

r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)
r.gStyle.SetLegendBorderSize(0)

# --------------------------------------------------------------------
# 1. Define cross sections (pb) for each signal file
#    (Matching the naming pattern of your .root files or some convention)
# --------------------------------------------------------------------
xsec_map = {
    # e.g. if your ROOT file is named ".../test_Higgsino_100_50.root"
    #      then the key is "test_Higgsino_100_50"
    # Adjust these values to match your logs:

    "test_Higgsino_100_50_simple": 0.07668,
    "test_Higgsino_100_80_simple": 0.07298,
    "test_Higgsino_100_90_simple": 0.07148,
    "test_Higgsino_100_95_simple": 0.07070,
    "test_Higgsino_100_98_simple": 0.07031,
    "test_Higgsino_100_99_simple": 0.07023,
    "test_Higgsino_120_70_simple": 0.05649,
    "test_Higgsino_120_100_simple": 0.05303,
    "test_Higgsino_120_110_simple": 0.05168,
    "test_Higgsino_120_115_simple": 0.05096,
    "test_Higgsino_120_118_simple": 0.05059,
    "test_Higgsino_120_119_simple": 0.05045,
    "test_Higgsino_140_90_simple": 0.03699,
    "test_Higgsino_140_120_simple": 0.03331,
    "test_Higgsino_140_130_simple": 0.03194,
    "test_Higgsino_140_135_simple": 0.03131,
    "test_Higgsino_140_138_simple": 0.03087,
    "test_Higgsino_140_139_simple": 0.03071,
    "test_Higgsino_160_110_simple": 0.01806,
    "test_Higgsino_160_140_simple": 0.01411,
    "test_Higgsino_160_150_simple": 0.01271,
    "test_Higgsino_160_155_simple": 0.01201,
    "test_Higgsino_160_158_simple": 0.01163,
    "test_Higgsino_160_159_simple": 0.01149,
    "test_Higgsino_180_130_simple": 0.004725,
    "test_Higgsino_180_160_simple": 0.0009906,
    "test_Higgsino_180_170_simple": 0.0001484,
    "test_Higgsino_182_132_simple": 0.004181,
    "test_Higgsino_182_162_simple": 0.0005962,
    "test_Higgsino_182_172_simple": 0.000005557,
}

# --------------------------------------------------------------------
# 2. Define integrated luminosity (ab^-1) and convert to pb^-1
# --------------------------------------------------------------------
luminosity_ab = 1.0  # 1 ab^-1
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*_simple.root")
signal_files.sort()

# for sig_file in signal_files:
#     base_name = os.path.basename(sig_file)                   # e.g. 'test_Higgsino_100_90_simple.root'
#     base_no_ext = os.path.splitext(base_name)[0]             # e.g. 'test_Higgsino_100_90_simple'

#     # Extract mass parameters from filename
#     parts = base_name.split('_')
#     m_N2 = int(parts[2])   # e.g. 100 GeV (χ2⁰ mass)
#     m_N1 = int(parts[3])   # e.g. 90 GeV (χ1⁰ mass)
#     delta_m = m_N2 - m_N1  # Mass difference

#     # Check cross-section availability
#     if base_no_ext not in xsec_map:
#         print(f"WARNING: No cross section found for {base_no_ext}")
#         continue  # <-- Now properly inside the loop

#     sigma_pb = xsec_map[base_no_ext]
#     print(f"File: {sig_file}, cross section: {sigma_pb} pb")

# for idx, sig_file in enumerate(signal_files):
#     print(hist_photon_energy_BG)
#     print(f"Processing signal file: {sig_file}")

#     # --- Open the signal file + get TTree ---
#     infile = r.TFile(sig_file)
#     intree = infile.Get("outputTree")
#     N_gen_sig = intree.GetEntries()  # <-- DEFINE BEFORE USE

#     # --- Compute per-event weight ---
#     base_name = os.path.basename(sig_file)
#     base_no_ext = os.path.splitext(base_name)[0]
#     if base_no_ext not in xsec_map:
#         print(f"  WARNING: No cross section found for {base_no_ext}. Skipping.")
#         continue

#     sigma_pb = xsec_map[base_no_ext]
#     weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig  # Now safe
#     print(f"  Signal weight    : {weight_sig:.4e}")
    
#     # Add mass info to data_dict (you need to define data_dict here)
#     data_dict = {
#         "m_N2": m_N2,
#         "delta_m": delta_m,
#     }


for idx, sig_file in enumerate(signal_files):
    print(hist_photon_energy_BG)
    print(f"Processing signal file: {sig_file}")

    # --- Extract mass parameters from filename ---
    base_name = os.path.basename(sig_file)
    parts = base_name.split('_')
    m_N2 = int(parts[2])  # e.g., 100 GeV (χ2⁰ mass)
    m_N1 = int(parts[3])  # e.g., 50 GeV (χ1⁰ mass)
    delta_m = m_N2 - m_N1

    # --- Check cross-section availability ---
    base_no_ext = os.path.splitext(base_name)[0]
    if base_no_ext not in xsec_map:
        print(f"  WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue  # <-- Now properly inside the loop

    sigma_pb = xsec_map[base_no_ext]
    print(f"File: {sig_file}, cross section: {sigma_pb} pb")

    # --- Open the signal file + get TTree ---
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()

    # --- Compute per-event weight ---
    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig
    print(f"  Signal weight: {weight_sig:.4e}")
    
    print(f"File: {sig_file}, cross section: {sigma_pb} pb")
    # ... proceed with TFile opening, weighting, histogram filling, etc.

# --------------------------------------------------------------------
# 3. Fill the BACKGROUND histograms (with background weight)
# --------------------------------------------------------------------

hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_Z_BG = r.TH1F("m_Z_BG",      "Z Mass (Background)",      100, 50, 250)
hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)

BG_sigma_pb = 2.641  # from your mention, e.g. 2.641 pb
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root")
BGtree = BGfile.Get("outputTree")
    
N_gen_bg = BGtree.GetEntries()
weight_bg = (BG_sigma_pb * luminosity_pb_inv) / N_gen_bg

print(f"Background cross section: {BG_sigma_pb} pb")
print(f"Luminosity: {luminosity_ab} ab^-1  (== {luminosity_pb_inv} pb^-1)")
print(f"N_gen (BG): {N_gen_bg}")
print(f"Background weight: {weight_bg:.4e}\n")


hist_H_BG.Sumw2()
hist_Z_BG.Sumw2()
hist_photon_energy_BG.Sumw2()

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

for event in BGtree:
    # Fill photon energies with the BG weight
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy, weight_bg)

    # Example: require exactly 2 muons
    if len(event.mu_pt) != 2:
        continue

    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M(), weight_bg)

    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M(), weight_bg)

#BGfile.Close()
print(hist_photon_energy_BG)

# --------------------------------------------------------------------
# 4. Process all SIGNAL files in the directory
# --------------------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*.root")
signal_files.sort()

for idx, sig_file in enumerate(signal_files):
    print(hist_photon_energy_BG)
    print(f"Processing signal file: {sig_file}")
    # --- Open the signal file + get TTree ---
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()  # <-- DEFINE BEFORE USE
    
    # --- Figure out which cross section to use ---
    # e.g. basename "test_Higgsino_100_50.root" => key "test_Higgsino_100_50"
    base_name = os.path.basename(sig_file)
    
    parts = base_name.split('_')
    m_N2 = int(parts[2])  # e.g., 100 GeV (χ2⁰ mass)
    m_N1 = int(parts[3])  # e.g., 50 GeV (χ1⁰ mass)
    delta_m = m_N2 - m_N1
    
    base_no_ext = os.path.splitext(base_name)[0]  # "test_Higgsino_100_50"    
    if base_no_ext not in xsec_map:
        print(f"  WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue

    sigma_pb = xsec_map[base_no_ext]
    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig  # Now safe

    
    print(hist_photon_energy_BG)


    # --- Compute per-event weight ---
    # weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig
    print(f"  Cross section    : {sigma_pb} pb")
    print(f"  N_gen (signal)   : {N_gen_sig}")
    print(f"  Signal weight    : {weight_sig:.4e}")

    # Create unique histograms
    hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}",      "Z Mass (Signal)",      100, 50, 250)
    hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)

    hist_H_signal.Sumw2()
    hist_Z_signal.Sumw2()
    hist_photon_energy_signal.Sumw2()
    print(hist_photon_energy_BG)

    # # --- Open the signal file + get TTree ---
    # infile = r.TFile(sig_file)
    # intree = infile.Get("outputTree")
    # N_gen_sig = intree.GetEntries()
    # # --- Compute per-event weight ---
    # weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig  # Now N_gen_sig is defined
    # print(f"  Cross section    : {sigma_pb} pb")
    # print(f"  N_gen (signal)   : {N_gen_sig}")

    # --- Fill signal hists with signal weight ---
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy, weight_sig)

        if len(event.mu_pt) != 2:
            continue

        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M(), weight_sig)

        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M(), weight_sig)
    infile.Close()

    print(hist_photon_energy_BG)
    # # --- Draw + compare with the pre-filled background hists ---
    # canvas_recoil = r.TCanvas(f"m_recoil_canvas_{idx}", f"Recoil Mass Canvas {idx}", 800, 600)
    # hist_H_signal.SetLineColor(r.kRed)
    # hist_H_signal.Draw("hist")
    # # draw the *already-filled* BG hists
    # hist_H_BG.SetLineColor(r.kYellow)
    # hist_H_BG.Draw("hist same")

    # hist_Z_signal.SetLineColor(r.kBlue)
    # hist_Z_signal.Draw("hist same")
    # hist_Z_BG.SetLineColor(r.kOrange)
    # hist_Z_BG.Draw("hist same")

    # legend = r.TLegend(0.6, 0.7, 0.85, 0.85)
    # legend.AddEntry(hist_H_signal, f"Recoil Mass (Signal {idx})", "l")
    # legend.AddEntry(hist_H_BG,     "Recoil Mass (BG)",           "l")
    # legend.AddEntry(hist_Z_signal, "Z Mass (Signal)",            "l")
    # legend.AddEntry(hist_Z_BG,     "Z Mass (BG)",                "l")
    # legend.Draw()

    # canvas_recoil.SaveAs(f"Recoil_Z_plots_{idx}.png")

    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)

    hist_photon_energy_signal.SetLineColor(r.kGreen)
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")
    maxbin=max(hist_photon_energy_BG.GetBinContent(hist_photon_energy_BG.GetMaximumBin()),
               hist_photon_energy_signal.GetBinContent(hist_photon_energy_signal.GetMaximumBin()))
    hist_photon_energy_signal.GetYaxis().SetRangeUser(0.5,10*maxbin)
    canvas_photon_E.SetLogy(1)
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # Write histograms to a ROOT file
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Also store the BG hists if you want them in the same file
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # Optionally, export to JSON
    data_dict = {
        "m_N2": m_N2,
        "delta_m": delta_m,
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }
    
    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }
    
    for hname, th1 in hist_map.items():
        nbins = th1.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = th1.GetXaxis().GetBinCenter(bin_num)
            bin_content = th1.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)
    
    with open(f"extracted_data_{idx}.json", "w") as f:
        json.dump(data_dict, f, indent=4)

    infile.Close()

print("\nAll signal files processed.")


ImportError: Failed to import libcppyy3_8. Please check that ROOT has been built for Python 3.8

In [12]:
import sys
import os
import math
import glob
import json
import ROOT as r

r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)
r.gStyle.SetLegendBorderSize(0)

# --------------------------------------------------------------------
# 1. Define cross sections (pb) for each signal file
# --------------------------------------------------------------------
xsec_map = {
    "test_Higgsino_100_50_simple": 0.07668,
    "test_Higgsino_100_80_simple": 0.07298,
    "test_Higgsino_100_90_simple": 0.07148,
    "test_Higgsino_100_95_simple": 0.07070,
    "test_Higgsino_100_98_simple": 0.07031,
    "test_Higgsino_100_99_simple": 0.07023,
    "test_Higgsino_120_70_simple": 0.05649,
    "test_Higgsino_120_100_simple": 0.05303,
    "test_Higgsino_120_110_simple": 0.05168,
    "test_Higgsino_120_115_simple": 0.05096,
    "test_Higgsino_120_118_simple": 0.05059,
    "test_Higgsino_120_119_simple": 0.05045,
    "test_Higgsino_140_90_simple": 0.03699,
    "test_Higgsino_140_120_simple": 0.03331,
    "test_Higgsino_140_130_simple": 0.03194,
    "test_Higgsino_140_135_simple": 0.03131,
    "test_Higgsino_140_138_simple": 0.03087,
    "test_Higgsino_140_139_simple": 0.03071,
    "test_Higgsino_160_110_simple": 0.01806,
    "test_Higgsino_160_140_simple": 0.01411,
    "test_Higgsino_160_150_simple": 0.01271,
    "test_Higgsino_160_155_simple": 0.01201,
    "test_Higgsino_160_158_simple": 0.01163,
    "test_Higgsino_160_159_simple": 0.01149,
    "test_Higgsino_180_130_simple": 0.004725,
    "test_Higgsino_180_160_simple": 0.0009906,
    "test_Higgsino_180_170_simple": 0.0001484,
    "test_Higgsino_182_132_simple": 0.004181,
    "test_Higgsino_182_162_simple": 0.0005962,
    "test_Higgsino_182_172_simple": 0.000005557,
}

# --------------------------------------------------------------------
# 2. Define integrated luminosity (ab^-1) and convert to pb^-1
# --------------------------------------------------------------------
luminosity_ab = 1.0  # 1 ab^-1
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

# --------------------------------------------------------------------
# 3. Fill the BACKGROUND histograms (with background weight)
# --------------------------------------------------------------------


hist_H_BG = ROOT.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_H_BG.SetDirectory(0)
hist_Z_BG = ROOT.TH1F("m_Z_BG", "Z Mass (Background)", 100, 50, 250)
hist_Z_BG.SetDirectory(0)
hist_photon_energy_BG = ROOT.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)
hist_photon_energy_BG.SetDirectory(0)



hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_Z_BG = r.TH1F("m_Z_BG",      "Z Mass (Background)",      100, 50, 250)
hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)

hist_H_BG.Sumw2()
hist_Z_BG.Sumw2()
hist_photon_energy_BG.Sumw2()

BG_sigma_pb = 2.641  # from your mention, e.g. 2.641 pb
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root")
BGtree = BGfile.Get("outputTree")

N_gen_bg = BGtree.GetEntries()
weight_bg = (BG_sigma_pb * luminosity_pb_inv) / N_gen_bg

print(f"Background cross section: {BG_sigma_pb} pb")
print(f"Luminosity: {luminosity_ab} ab^-1  (== {luminosity_pb_inv} pb^-1)")
print(f"N_gen (BG): {N_gen_bg}")
print(f"Background weight: {weight_bg:.4e}\n")

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

for event in BGtree:
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy, weight_bg)

    if len(event.mu_pt) != 2:
        continue

    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M(), weight_bg)

    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M(), weight_bg)

BGfile.Close()

print("Finished filling background. Now processing signal...")

# --------------------------------------------------------------------
# 4. Process all SIGNAL files in the directory
# --------------------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*_simple.root")
signal_files.sort()

for idx, sig_file in enumerate(signal_files):
    print("\n===================================================")
    print(f"Processing signal file #{idx}: {sig_file}")
    
    base_name = os.path.basename(sig_file)
    base_no_ext = os.path.splitext(base_name)[0]

    # Parse masses from filename, e.g. 'test_Higgsino_100_50_simple.root'
    parts = base_name.split('_')
    m_N2 = int(parts[2])  # e.g., 100
    m_N1 = int(parts[3])  # e.g., 50
    delta_m = m_N2 - m_N1

    if base_no_ext not in xsec_map:
        print(f"WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue

    sigma_pb = xsec_map[base_no_ext]
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()

    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig

    print(f"  Cross section    : {sigma_pb} pb")
    print(f"  N_gen (signal)   : {N_gen_sig}")
    print(f"  Signal weight    : {weight_sig:.4e}")
    print(f"  Masses => N2={m_N2} GeV, N1={m_N1} GeV, deltaM={delta_m} GeV")

    # Create unique histograms for this signal
    # hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    # hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}",      "Z Mass (Signal)",      100, 50, 250)
    # hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)
    # hist_photon_energy_signal.SetDirectory(0)



    hist_H_signal = ROOT.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_H_signal.SetDirectory(0)
    hist_Z_signal = ROOT.TH1F(f"m_Z_signal_{idx}", "Z Mass (Signal)", 100, 50, 250)
    hist_Z_signal.SetDirectory(0)
    hist_photon_energy_signal = ROOT.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)
    hist_photon_energy_signal.SetDirectory(0)
    
    hist_H_signal.Sumw2()
    hist_Z_signal.Sumw2()
    hist_photon_energy_signal.Sumw2()
    # Fill signal hists with the computed weight
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy, weight_sig)

        if len(event.mu_pt) != 2:
            continue

        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M(), weight_sig)

        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M(), weight_sig)

    infile.Close()




    # Compare signal vs. background in a Photon Energy plot
    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)
    hist_photon_energy_signal.SetLineColor(r.kGreen)
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")
    maxbin = max(
        hist_photon_energy_BG.GetBinContent(hist_photon_energy_BG.GetMaximumBin()),
        hist_photon_energy_signal.GetBinContent(hist_photon_energy_signal.GetMaximumBin())
    )
    hist_photon_energy_signal.GetYaxis().SetRangeUser(0.5, 10*maxbin)
    canvas_photon_E.SetLogy(1)
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # Write histograms to a ROOT file
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Also store the BG hists if you want them in the same file
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # Optionally, export to JSON
    data_dict = {
        "m_N2": m_N2,
        "delta_m": delta_m,
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }

    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }

    for hname, th1 in hist_map.items():
        nbins = th1.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = th1.GetXaxis().GetBinCenter(bin_num)
            bin_content = th1.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)

    with open(f"extracted_data_{idx}.json", "w") as jf:
        json.dump(data_dict, jf, indent=4)

print("\nAll signal files processed.")


ImportError: Failed to import libcppyy3_8. Please check that ROOT has been built for Python 3.8

In [13]:
root-config --python-version


NameError: name 'root' is not defined

In [14]:
import sys
import os
import math
import glob
import json
import ROOT as r

r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)
r.gStyle.SetLegendBorderSize(0)

# --------------------------------------------------------------------
# 1. Define cross sections (pb) for each signal file
#    (Matching the naming pattern of your .root files or some convention)
# --------------------------------------------------------------------
xsec_map = {
    # e.g. if your ROOT file is named ".../test_Higgsino_100_50.root"
    #      then the key is "test_Higgsino_100_50"
    # Adjust these values to match your logs:

    "test_Higgsino_100_50_simple": 0.07668,
    "test_Higgsino_100_80_simple": 0.07298,
    "test_Higgsino_100_90_simple": 0.07148,
    "test_Higgsino_100_95_simple": 0.07070,
    "test_Higgsino_100_98_simple": 0.07031,
    "test_Higgsino_100_99_simple": 0.07023,
    "test_Higgsino_120_70_simple": 0.05649,
    "test_Higgsino_120_100_simple": 0.05303,
    "test_Higgsino_120_110_simple": 0.05168,
    "test_Higgsino_120_115_simple": 0.05096,
    "test_Higgsino_120_118_simple": 0.05059,
    "test_Higgsino_120_119_simple": 0.05045,
    "test_Higgsino_140_90_simple": 0.03699,
    "test_Higgsino_140_120_simple": 0.03331,
    "test_Higgsino_140_130_simple": 0.03194,
    "test_Higgsino_140_135_simple": 0.03131,
    "test_Higgsino_140_138_simple": 0.03087,
    "test_Higgsino_140_139_simple": 0.03071,
    "test_Higgsino_160_110_simple": 0.01806,
    "test_Higgsino_160_140_simple": 0.01411,
    "test_Higgsino_160_150_simple": 0.01271,
    "test_Higgsino_160_155_simple": 0.01201,
    "test_Higgsino_160_158_simple": 0.01163,
    "test_Higgsino_160_159_simple": 0.01149,
    "test_Higgsino_180_130_simple": 0.004725,
    "test_Higgsino_180_160_simple": 0.0009906,
    "test_Higgsino_180_170_simple": 0.0001484,
    "test_Higgsino_182_132_simple": 0.004181,
    "test_Higgsino_182_162_simple": 0.0005962,
    "test_Higgsino_182_172_simple": 0.000005557,
}

# --------------------------------------------------------------------
# 2. Define integrated luminosity (ab^-1) and convert to pb^-1
# --------------------------------------------------------------------
luminosity_ab = 1.0  # 1 ab^-1
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*_simple.root")
signal_files.sort()

for sig_file in signal_files:
    base_name = os.path.basename(sig_file)                   # e.g. 'test_Higgsino_100_90_simple.root'
    base_no_ext = os.path.splitext(base_name)[0]            # e.g. 'test_Higgsino_100_90_simple'
    # Remove '_simple' from the end:
    # base_no_ext = base_no_ext.replace('_simple', '')        # => 'test_Higgsino_100_90'

    if base_no_ext not in xsec_map:
        print(f"WARNING: No cross section found for {base_no_ext}")
        continue

    sigma_pb = xsec_map[base_no_ext]
    print(f"File: {sig_file}, cross section: {sigma_pb} pb")
    # ... proceed with TFile opening, weighting, histogram filling, etc.

# --------------------------------------------------------------------
# 3. Fill the BACKGROUND histograms (with background weight)
# --------------------------------------------------------------------

hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_Z_BG = r.TH1F("m_Z_BG",      "Z Mass (Background)",      100, 50, 250)
hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)

BG_sigma_pb = 2.641  # from your mention, e.g. 2.641 pb
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root")
BGtree = BGfile.Get("outputTree")

N_gen_bg = BGtree.GetEntries()
weight_bg = (BG_sigma_pb * luminosity_pb_inv) / N_gen_bg

print(f"Background cross section: {BG_sigma_pb} pb")
print(f"Luminosity: {luminosity_ab} ab^-1  (== {luminosity_pb_inv} pb^-1)")
print(f"N_gen (BG): {N_gen_bg}")
print(f"Background weight: {weight_bg:.4e}\n")


hist_H_BG.Sumw2()
hist_Z_BG.Sumw2()
hist_photon_energy_BG.Sumw2()

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

for event in BGtree:
    # Fill photon energies with the BG weight
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy, weight_bg)

    # Example: require exactly 2 muons
    if len(event.mu_pt) != 2:
        continue

    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M(), weight_bg)

    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M(), weight_bg)

#BGfile.Close()
print(hist_photon_energy_BG)

# --------------------------------------------------------------------
# 4. Process all SIGNAL files in the directory
# --------------------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*.root")
signal_files.sort()

for idx, sig_file in enumerate(signal_files):
    print(hist_photon_energy_BG)

    print(f"Processing signal file: {sig_file}")

    # --- Figure out which cross section to use ---
    # e.g. basename "test_Higgsino_100_50.root" => key "test_Higgsino_100_50"
    base_name = os.path.basename(sig_file)
    base_no_ext = os.path.splitext(base_name)[0]  # "test_Higgsino_100_50"
    if base_no_ext not in xsec_map:
        print(f"  WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue

    sigma_pb = xsec_map[base_no_ext]
    print(hist_photon_energy_BG)


    # --- Compute per-event weight ---
    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig
    print(f"  Cross section    : {sigma_pb} pb")
    print(f"  N_gen (signal)   : {N_gen_sig}")
    print(f"  Signal weight    : {weight_sig:.4e}")

    # Create unique histograms
    hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}",      "Z Mass (Signal)",      100, 50, 250)
    hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)

    hist_H_signal.Sumw2()
    hist_Z_signal.Sumw2()
    hist_photon_energy_signal.Sumw2()
    print(hist_photon_energy_BG)

    # --- Open the signal file + get TTree ---
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()

    # --- Fill signal hists with signal weight ---
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy, weight_sig)

        if len(event.mu_pt) != 2:
            continue

        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M(), weight_sig)

        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M(), weight_sig)
    infile.Close()

    print(hist_photon_energy_BG)
    # # --- Draw + compare with the pre-filled background hists ---
    # canvas_recoil = r.TCanvas(f"m_recoil_canvas_{idx}", f"Recoil Mass Canvas {idx}", 800, 600)
    # hist_H_signal.SetLineColor(r.kRed)
    # hist_H_signal.Draw("hist")
    # # draw the *already-filled* BG hists
    # hist_H_BG.SetLineColor(r.kYellow)
    # hist_H_BG.Draw("hist same")

    # hist_Z_signal.SetLineColor(r.kBlue)
    # hist_Z_signal.Draw("hist same")
    # hist_Z_BG.SetLineColor(r.kOrange)
    # hist_Z_BG.Draw("hist same")

    # legend = r.TLegend(0.6, 0.7, 0.85, 0.85)
    # legend.AddEntry(hist_H_signal, f"Recoil Mass (Signal {idx})", "l")
    # legend.AddEntry(hist_H_BG,     "Recoil Mass (BG)",           "l")
    # legend.AddEntry(hist_Z_signal, "Z Mass (Signal)",            "l")
    # legend.AddEntry(hist_Z_BG,     "Z Mass (BG)",                "l")
    # legend.Draw()

    # canvas_recoil.SaveAs(f"Recoil_Z_plots_{idx}.png")

    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)

    hist_photon_energy_signal.SetLineColor(r.kGreen)
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")
    maxbin=max(hist_photon_energy_BG.GetBinContent(hist_photon_energy_BG.GetMaximumBin()),
               hist_photon_energy_signal.GetBinContent(hist_photon_energy_signal.GetMaximumBin()))
    hist_photon_energy_signal.GetYaxis().SetRangeUser(0.5,10*maxbin)
    canvas_photon_E.SetLogy(1)
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # Write histograms to a ROOT file
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Also store the BG hists if you want them in the same file
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # Optionally, export to JSON
    data_dict = {
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }
    
    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }
    
    for hname, th1 in hist_map.items():
        nbins = th1.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = th1.GetXaxis().GetBinCenter(bin_num)
            bin_content = th1.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)
    
    with open(f"extracted_data_{idx}.json", "w") as f:
        json.dump(data_dict, f, indent=4)

    infile.Close()

print("\nAll signal files processed.")

ImportError: Failed to import libcppyy3_8. Please check that ROOT has been built for Python 3.8

In [3]:
import sys
import os
import math
import glob
import json
import ROOT as r

r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)
r.gStyle.SetLegendBorderSize(0)

# --------------------------------------------------------------------
# 1. Define cross sections (pb) for each signal file
# --------------------------------------------------------------------
xsec_map = {
    "test_Higgsino_100_50_simple": 0.07668,
    "test_Higgsino_100_80_simple": 0.07298,
    "test_Higgsino_100_90_simple": 0.07148,
    "test_Higgsino_100_95_simple": 0.07070,
    "test_Higgsino_100_98_simple": 0.07031,
    "test_Higgsino_100_99_simple": 0.07023,
    "test_Higgsino_120_70_simple": 0.05649,
    "test_Higgsino_120_100_simple": 0.05303,
    "test_Higgsino_120_110_simple": 0.05168,
    "test_Higgsino_120_115_simple": 0.05096,
    "test_Higgsino_120_118_simple": 0.05059,
    "test_Higgsino_120_119_simple": 0.05045,
    "test_Higgsino_140_90_simple": 0.03699,
    "test_Higgsino_140_120_simple": 0.03331,
    "test_Higgsino_140_130_simple": 0.03194,
    "test_Higgsino_140_135_simple": 0.03131,
    "test_Higgsino_140_138_simple": 0.03087,
    "test_Higgsino_140_139_simple": 0.03071,
    "test_Higgsino_160_110_simple": 0.01806,
    "test_Higgsino_160_140_simple": 0.01411,
    "test_Higgsino_160_150_simple": 0.01271,
    "test_Higgsino_160_155_simple": 0.01201,
    "test_Higgsino_160_158_simple": 0.01163,
    "test_Higgsino_160_159_simple": 0.01149,
    "test_Higgsino_180_130_simple": 0.004725,
    "test_Higgsino_180_160_simple": 0.0009906,
    "test_Higgsino_180_170_simple": 0.0001484,
    "test_Higgsino_182_132_simple": 0.004181,
    "test_Higgsino_182_162_simple": 0.0005962,
    "test_Higgsino_182_172_simple": 0.000005557,
}

# --------------------------------------------------------------------
# 2. Define integrated luminosity (ab^-1) and convert to pb^-1
# --------------------------------------------------------------------
luminosity_ab = 1.0  # 1 ab^-1
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

# --------------------------------------------------------------------
# 3. Fill the BACKGROUND histograms (with background weight)
# --------------------------------------------------------------------
hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_Z_BG = r.TH1F("m_Z_BG",      "Z Mass (Background)",      100, 50, 250)
hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)

hist_H_BG.Sumw2()
hist_Z_BG.Sumw2()
hist_photon_energy_BG.Sumw2()

BG_sigma_pb = 2.641  # from your mention, e.g. 2.641 pb
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root")
BGtree = BGfile.Get("outputTree")

N_gen_bg = BGtree.GetEntries()
weight_bg = (BG_sigma_pb * luminosity_pb_inv) / N_gen_bg

print(f"Background cross section: {BG_sigma_pb} pb")
print(f"Luminosity: {luminosity_ab} ab^-1  (== {luminosity_pb_inv} pb^-1)")
print(f"N_gen (BG): {N_gen_bg}")
print(f"Background weight: {weight_bg:.4e}\n")

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

for event in BGtree:
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy, weight_bg)

    if len(event.mu_pt) != 2:
        continue

    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M(), weight_bg)

    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M(), weight_bg)

BGfile.Close()

print("Finished filling background. Now processing signal...")

# --------------------------------------------------------------------
# 4. Process all SIGNAL files in the directory
# --------------------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*_simple.root")
signal_files.sort()

for idx, sig_file in enumerate(signal_files):
    print("\n===================================================")
    print(f"Processing signal file #{idx}: {sig_file}")
    
    base_name = os.path.basename(sig_file)
    base_no_ext = os.path.splitext(base_name)[0]

    # Parse masses from filename, e.g. 'test_Higgsino_100_50_simple.root'
    parts = base_name.split('_')
    m_N2 = int(parts[2])  # e.g., 100
    m_N1 = int(parts[3])  # e.g., 50
    delta_m = m_N2 - m_N1

    if base_no_ext not in xsec_map:
        print(f"WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue

    sigma_pb = xsec_map[base_no_ext]
    infile = r.TFile(sig_file)
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()

    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig

    print(f"  Cross section    : {sigma_pb} pb")
    print(f"  N_gen (signal)   : {N_gen_sig}")
    print(f"  Signal weight    : {weight_sig:.4e}")
    print(f"  Masses => N2={m_N2} GeV, N1={m_N1} GeV, deltaM={delta_m} GeV")

    # Create unique histograms for this signal
    hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}",      "Z Mass (Signal)",      100, 50, 250)
    hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)

    hist_H_signal.Sumw2()
    hist_Z_signal.Sumw2()
    hist_photon_energy_signal.Sumw2()

    # Fill signal hists with the computed weight
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy, weight_sig)

        if len(event.mu_pt) != 2:
            continue

        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M(), weight_sig)

        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M(), weight_sig)


    # Compare signal vs. background in a Photon Energy plot
    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)
    hist_photon_energy_signal.SetLineColor(r.kGreen)
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")
    maxbin = max(
        hist_photon_energy_BG.GetBinContent(hist_photon_energy_BG.GetMaximumBin()),
        hist_photon_energy_signal.GetBinContent(hist_photon_energy_signal.GetMaximumBin())
    )
    hist_photon_energy_signal.GetYaxis().SetRangeUser(0.5, 10*maxbin)
    canvas_photon_E.SetLogy(1)
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # Write histograms to a ROOT file
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Also store the BG hists if you want them in the same file
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # Optionally, export to JSON
    data_dict = {
        "m_N2": m_N2,
        "delta_m": delta_m,
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }

    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }

    for hname, th1 in hist_map.items():
        nbins = th1.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = th1.GetXaxis().GetBinCenter(bin_num)
            bin_content = th1.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)

    with open(f"extracted_data_{idx}.json", "w") as jf:
        json.dump(data_dict, jf, indent=4)

    infile.Close()

print("\nAll signal files processed.")


Background cross section: 2.641 pb
Luminosity: 1.0 ab^-1  (== 1000000.0 pb^-1)
N_gen (BG): 10000
Background weight: 2.6410e+02

Finished filling background. Now processing signal...

Processing signal file #0: /data/mhance/FCCee/test_Higgsino_100_50_simple.root
  Cross section    : 0.07668 pb
  N_gen (signal)   : 10000
  Signal weight    : 7.6680e+00
  Masses => N2=100 GeV, N1=50 GeV, deltaM=50 GeV

Processing signal file #1: /data/mhance/FCCee/test_Higgsino_100_80_simple.root
  Cross section    : 0.07298 pb
  N_gen (signal)   : 10000
  Signal weight    : 7.2980e+00
  Masses => N2=100 GeV, N1=80 GeV, deltaM=20 GeV

Processing signal file #2: /data/mhance/FCCee/test_Higgsino_100_90_simple.root
  Cross section    : 0.07148 pb
  N_gen (signal)   : 9997
  Signal weight    : 7.1501e+00
  Masses => N2=100 GeV, N1=90 GeV, deltaM=10 GeV

Processing signal file #3: /data/mhance/FCCee/test_Higgsino_100_95_simple.root
  Cross section    : 0.0707 pb
  N_gen (signal)   : 9980
  Signal weight    : 7

Warning in <TROOT::Append>: Replacing existing TH1: m_recoil_BG (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: m_Z_BG (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ph_E_BG (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: photon_energy_canvas_0
Info in <TCanvas::Print>: png file PhotonEnergy_plots_0.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_1.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_2.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_3.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_4.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_5.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_6.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_7.png has been created
Info in <TCanvas::Print>: png file P